In [40]:
import tensorflow as tf
import base64
import boto3
import time
import json
import random

client = boto3.client('sagemaker-runtime')
endpoint_name = 'tensorflow-inference-2022-10-25-08-29-10-875'
content_type = "application/json"
a = tf.data.TFRecordDataset("part-r-00000")

In [6]:
example = []
for i in a:
    tmp = i
    example.append(i)
    if len(example) == 1:
        break

In [9]:
varlen_features_text = "city,click_url_tag_series_avg_diff,click_url_tag_series_cnt,click_url_tag_series_first_diff,click_url_tag_series_first_last_diff,click_url_tag_series_first_second_diff,click_url_tag_series_last_diff,click_url_tag_series_last_second_diff,click_url_tag_series_orderd_by_cnt,click_url_tag_series_orderd_by_time,click_url_tag_series_rate,click_url_tag_series_second_last_diff,collection_id,country,ctx_item_id,cxr_14day,cxr_1day,cxr_30day,cxr_3day,cxr_7day,day_of_month,day_of_week,hour_of_day,hour_slice,is_weekend,item_id,item_lda_v1,price_and_3diff,province,query,stat_ctx_event_14day,stat_ctx_event_1day,stat_ctx_event_30day,stat_ctx_event_3day,stat_ctx_event_7day,stat_event_14day,stat_event_1day,stat_event_30day,stat_event_3day,stat_event_7day,stat_search_14day,stat_search_1day,stat_search_30day,stat_search_3day,stat_search_7day,store_id,title_tags,ui_lda_sim,user_lda_v1"
varlen_features = varlen_features_text.split(",")

In [12]:
feature_description = {}
feature_description.update(
    {k + "_index": tf.io.VarLenFeature(dtype=tf.int64) for k in varlen_features}
)
feature_description.update(
    {k + "_value": tf.io.VarLenFeature(dtype=tf.float32) for k in varlen_features}
)

In [29]:
feature_description = {}
feature_description.update(
    {k + "_index":_int64_feature([1,2,3,4,5]) for k in varlen_features}
)
feature_description.update(
    {k + "_value":_float_feature([1.1,2.2,3.3,4.4,5.5]) for k in varlen_features}
)

In [21]:
for raw_record in a.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "city_index"
    value {
      int64_list {
        value: 238277
      }
    }
  }
  feature {
    key: "city_value"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "click_url_tag_series_avg_diff_index"
    value {
      int64_list {
        value: 431721
        value: 241763
        value: 306433
        value: 479024
        value: 200080
        value: 339330
        value: 289485
        value: 382908
        value: 321301
        value: 229531
        value: 298099
        value: 431921
        value: 319594
      }
    }
  }
  feature {
    key: "click_url_tag_series_avg_diff_value"
    value {
      float_list {
        value: 3.450870644883253e-05
        value: 3.450870644883253e-05
        value: 3.450870644883253e-05
        value: 3.450870644883253e-05
        value: 3.450870644883253e-05
        value: 3.450870644883253e-05
        value: 8.098303078440949e-05
        value: 3.450870644883253e-05
  

In [25]:
example.features

feature {
  key: "city_index"
  value {
    int64_list {
      value: 238277
    }
  }
}
feature {
  key: "city_value"
  value {
    float_list {
      value: 1.0
    }
  }
}
feature {
  key: "click_url_tag_series_avg_diff_index"
  value {
    int64_list {
      value: 431721
      value: 241763
      value: 306433
      value: 479024
      value: 200080
      value: 339330
      value: 289485
      value: 382908
      value: 321301
      value: 229531
      value: 298099
      value: 431921
      value: 319594
    }
  }
}
feature {
  key: "click_url_tag_series_avg_diff_value"
  value {
    float_list {
      value: 3.450870644883253e-05
      value: 3.450870644883253e-05
      value: 3.450870644883253e-05
      value: 3.450870644883253e-05
      value: 3.450870644883253e-05
      value: 3.450870644883253e-05
      value: 8.098303078440949e-05
      value: 3.450870644883253e-05
      value: 1.6987034541671164e-05
      value: 1.6987034541671164e-05
      value: 1.6987034541671164e-05
 

In [26]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [41]:
def serialize_example(num):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature_description = {}
    feature_description.update(
        {k + "_index":_int64_feature([i for i in range(num)]) for k in varlen_features}
    )
    feature_description.update(
        {k + "_value":_float_feature([random.random() for i in range(num)]) for k in varlen_features}
    )

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature_description))
    return example_proto.SerializeToString()

In [50]:
tmp1 = serialize_example(10)
body1 = base64.b64encode(tmp1).decode()
tmp2 = serialize_example(1)
body2 = base64.b64encode(tmp2).decode()
tmp3 = serialize_example(11)
body3 = base64.b64encode(tmp3).decode()
tmp4 = serialize_example(15)
body4 = base64.b64encode(tmp4).decode()
tmp5 = serialize_example(3)
body5 = base64.b64encode(tmp5).decode()

In [51]:
input_body = {'signature_name': 'predict',
             'instances':[{'examples':{
                 'b64':body1
             }},
                          {'examples':{
                 'b64':body2
             }},
                          {'examples':{
                 'b64':body3
             }},
                          {'examples':{
                 'b64':body4
             }}
                          ,
                          {'examples':{
                 'b64':body5
             }}
                         
                         ]}
response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        # Accept=accept,
        Body = json.dumps(input_body)
        )
out = (json.loads(response['Body'].read()))

In [52]:
for i in out['predictions']:
    print(i)

{'class_ids': [1], 'classes': ['1'], 'all_class_ids': [0, 1], 'logistic': [0.585407495], 'all_classes': ['0', '1'], 'probabilities': [0.414592594, 0.585407495], 'logits': [0.34501186]}
{'class_ids': [0], 'classes': ['0'], 'all_class_ids': [0, 1], 'logistic': [0.0420969352], 'all_classes': ['0', '1'], 'probabilities': [0.957903, 0.0420969352], 'logits': [-3.1247716]}
{'class_ids': [1], 'classes': ['1'], 'all_class_ids': [0, 1], 'logistic': [0.664108396], 'all_classes': ['0', '1'], 'probabilities': [0.335891634, 0.664108396], 'logits': [0.681656837]}
{'class_ids': [0], 'classes': ['0'], 'all_class_ids': [0, 1], 'logistic': [0.000524788164], 'all_classes': ['0', '1'], 'probabilities': [0.999475181, 0.000524788164], 'logits': [-7.55199099]}
{'class_ids': [1], 'classes': ['1'], 'all_class_ids': [0, 1], 'logistic': [0.556003332], 'all_classes': ['0', '1'], 'probabilities': [0.443996668, 0.556003332], 'logits': [0.224957258]}
